In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 5728, done.
remote: Counting objects: 100% (917/917), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 5728 (delta 793), reused 806 (delta 693), pack-reused 4811
Receiving objects: 100% (5728/5728), 2.87 GiB | 27.83 MiB/s, done.
Resolving deltas: 100% (4619/4619), done.
Updating files: 100% (487/487), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 1.0 MB/s eta 0:00:00


In [3]:
# ! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_lightgcn_pytorch.db',
    'copy_prev_best_params': False,
    'save_github': True
}

In [10]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_lightgcn_pytorch.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

https://www.kaggle.com/code/dipanjandas96/lightgcn-pytorch-from-scratch

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


class LightGCN_Model(nn.Module):
    def __init__(self, data, n_users, n_items, n_layers, latent_dim):
        super(LightGCN_Model, self).__init__()
        self.data = data
        self.n_users = n_users
        self.n_items = n_items
        self.n_layers = n_layers
        self.latent_dim = latent_dim
        self.init_embedding()
        self.norm_adj_mat_sparse_tensor = self.get_A_tilda()

    def init_embedding(self):
        self.E0 = nn.Embedding(self.n_users + self.n_items, self.latent_dim)
        nn.init.xavier_uniform_(self.E0.weight)
        self.E0.weight = nn.Parameter(self.E0.weight)

    def get_A_tilda(self):
        R = sps.dok_matrix((self.n_users, self.n_items), dtype = np.float32)
        R[self.data['UserID'], self.data['ItemID']] = 1.0

        adj_mat = sps.dok_matrix(
                (self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32
            )
        adj_mat = adj_mat.tolil()
        R = R.tolil()

        adj_mat[:self.n_users, self.n_users:] = R
        adj_mat[self.n_users:, :self.n_users] = R.T
        adj_mat = adj_mat.todok()

        rowsum = np.array(adj_mat.sum(1))
        d_inv = np.power(rowsum + 1e-9, -0.5).flatten()
        d_inv[np.isinf(d_inv)] = 0.0
        d_mat_inv = sps.diags(d_inv)
        norm_adj_mat = d_mat_inv.dot(adj_mat)
        norm_adj_mat = norm_adj_mat.dot(d_mat_inv)
        
        # Below Code is toconvert the dok_matrix to sparse tensor.
        
        norm_adj_mat_coo = norm_adj_mat.tocoo().astype(np.float32)
        values = norm_adj_mat_coo.data
        indices = np.vstack((norm_adj_mat_coo.row, norm_adj_mat_coo.col))

        i = torch.LongTensor(indices)
        v = torch.FloatTensor(values)
        shape = norm_adj_mat_coo.shape

        norm_adj_mat_sparse_tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))

        return norm_adj_mat_sparse_tensor
    
    def propagate_through_layers(self):
        all_layer_embedding = [self.E0.weight]
        E_lyr = self.E0.weight

        for layer in range(self.n_layers):
            E_lyr = torch.sparse.mm(self.norm_adj_mat_sparse_tensor, E_lyr)
            all_layer_embedding.append(E_lyr)

        all_layer_embedding = torch.stack(all_layer_embedding)
        mean_layer_embedding = torch.mean(all_layer_embedding, axis = 0)

        final_user_Embed, final_item_Embed = torch.split(mean_layer_embedding, [self.n_users, self.n_items])
        initial_user_Embed, initial_item_Embed = torch.split(self.E0.weight, [self.n_users, self.n_items])

        return final_user_Embed, final_item_Embed, initial_user_Embed, initial_item_Embed

    def forward(self, users, pos_items, neg_items):
        final_user_Embed, final_item_Embed, initial_user_Embed, initial_item_Embed = self.propagate_through_layers()

        users_emb, pos_emb, neg_emb = final_user_Embed[users], final_item_Embed[pos_items], final_item_Embed[neg_items]
        userEmb0,  posEmb0, negEmb0 = initial_user_Embed[users], initial_item_Embed[pos_items], initial_item_Embed[neg_items]

        return users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0

In [15]:
class LightGCNRecommender_PyTorch(BaseMatrixFactorizationRecommender, Incremental_Training_Early_Stopping):
    
    RECOMMENDER_NAME = "LightGCNRecommender_PyTorch"
    AVAILABLE_OPTIMIZERS = ["adam", "sgd", "rmsprop", "adagrad", "adadelta"]
    
    def __init__(self, URM_train):
        super(LightGCNRecommender_PyTorch, self).__init__(URM_train)
        
        # building the tensorflow sparse tensor, .T is done because an array of (row, col) is needed
        URM_train_coo = self.URM_train.tocoo()
        self.URM_train_df = pd.DataFrame({'UserID': URM_train_coo.row, 'ItemID': URM_train_coo.col, 'Rating': URM_train_coo.data})
        
        self.interacted_items_df = self.URM_train_df.groupby('UserID')['ItemID'].apply(list).reset_index()
        self.warm_user_ids = self.interacted_items_df.UserID.values
        
    def fit(
        self, 
        epochs=100, 
        num_layers=2,
        embedding_size=10,
        learning_rate=1e-3,
        batch_size=64,
        optimizer='adam',
        l2_reg=1e-2,
        **earlystopping_kwargs
    ):
        
        if optimizer not in self.AVAILABLE_OPTIMIZERS:
            raise ValueError("Value for 'optimizer' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_OPTIMIZERS, optimizer))

        self.num_layers = num_layers
        self.embedding_size = embedding_size
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.l2_reg = l2_reg
        
        self.lightGCN_model = LightGCN_Model(self.URM_train_df, self.n_users, self.n_items, self.num_layers, self.embedding_size)
        
        self._print('Built LightGCN Model, starting to train')
        
        self._build_optimizer(optimizer)
        
        self._train_with_early_stopping(epochs,
                                        algorithm_name = self.RECOMMENDER_NAME,
                                        **earlystopping_kwargs)

        self.USER_factors = self.USER_factors_best
        self.ITEM_factors = self.ITEM_factors_best
        
    def _build_optimizer(self, optimizer):
        if optimizer == 'adam':
            self.optimizer = torch.optim.Adam(self.lightGCN_model.parameters(), lr=self.learning_rate)
        elif optimizer == 'rmsprop':
            self.optimizer = torch.optim.RMSprop(self.lightGCN_model.parameters(), lr=self.learning_rate)
        elif optimizer == 'sgd':
            self.optimizer = torch.optim.SGD(self.lightGCN_model.parameters(), lr=self.learning_rate)
        elif optimizer == 'adagrad':
            self.optimizer = torch.optim.Adagrad(self.lightGCN_model.parameters(), lr=self.learning_rate)
        else:
            self.optimizer = torch.optim.Adadelta(self.lightGCN_model.parameters(), lr=self.learning_rate)
            

    def _run_epoch(self, num_epoch):
        
        n_batch = int(len(self.warm_user_ids) / self.batch_size)
  
        self.lightGCN_model.train()
        for batch_idx in tqdm.auto.tqdm(range(n_batch)):

            self.optimizer.zero_grad()

            users, pos_items, neg_items = self._data_loader()

            users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0 = self.lightGCN_model.forward(users, pos_items, neg_items)

            mf_loss, reg_loss = self._bpr_loss(users, users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0)
            reg_loss = self.l2_reg * reg_loss
            final_loss = mf_loss + reg_loss

            final_loss.backward()
            self.optimizer.step()
            
        embeddings = self.lightGCN_model.E0.weight.detach().numpy()
        self.USER_factors = embeddings[:self.n_users, :]
        self.ITEM_factors = embeddings[self.n_users:, :]
        
    def _update_best_model(self):
        self.USER_factors_best = self.USER_factors.copy()
        self.ITEM_factors_best = self.ITEM_factors.copy()
        
    def _prepare_model_for_validation(self):
        pass
    
    def _bpr_loss(self, users, users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0):
  
        reg_loss = (1/2)*(userEmb0.norm().pow(2) + 
                        posEmb0.norm().pow(2)  +
                        negEmb0.norm().pow(2))/float(len(users))
        pos_scores = torch.mul(users_emb, pos_emb)
        pos_scores = torch.sum(pos_scores, dim=1)
        neg_scores = torch.mul(users_emb, neg_emb)
        neg_scores = torch.sum(neg_scores, dim=1)

        loss = torch.mean(torch.nn.functional.softplus(neg_scores - pos_scores))

        return loss, reg_loss
    
    def _data_loader(self):

        def sample_neg(x):
            while True:
                neg_id = random.randint(0, self.n_items - 1)
                if neg_id not in x:
                    return neg_id

        if self.n_users < self.batch_size:
            indices = [x for x in range(len(self.warm_user_ids))]
            users = [self.warm_user_ids[random.choice(indices)] for _ in range(self.batch_size)]
        else:
            users = random.sample(self.warm_user_ids.tolist(), self.batch_size)

        users.sort()
        users_df = pd.DataFrame(users, columns = ['users'])

        interacted_items_df = pd.merge(self.interacted_items_df, users_df, how = 'right', left_on = 'UserID', right_on = 'users')

        pos_items = interacted_items_df['ItemID'].apply(lambda x: random.choice(x)).values

        neg_items = interacted_items_df['ItemID'].apply(lambda x: sample_neg(x)).values

        return list(users), list(pos_items), list(neg_items)

In [16]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

early_stopping_params = {
    "validation_every_n": 1,
    "stop_on_validation": True,
    "evaluator_object": evaluator,
    "lower_validations_allowed": 3,
    "validation_metric": "MAP",
    "epochs": 10
} 

recommender = LightGCNRecommender_PyTorch(URM_train)
recommender.fit(batch_size=128, **early_stopping_params)

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
LightGCNRecommender_PyTorch: URM Detected 627 ( 4.8%) users with no interactions.
LightGCNRecommender_PyTorch: URM Detected 247 ( 1.1%) items with no interactions.
LightGCNRecommender_PyTorch: Built LightGCN Model, starting to train


  0%|          | 0/96 [00:00<?, ?it/s]

LightGCNRecommender_PyTorch: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 10.31 sec. Users per second: 1015
LightGCNRecommender_PyTorch: CUTOFF: 10 - PRECISION: 0.0030604, PRECISION_RECALL_MIN_DEN: 0.0045021, RECALL: 0.0033886, MAP: 0.0009684, MAP_MIN_DEN: 0.0014267, MRR: 0.0092633, NDCG: 0.0038295, F1: 0.0032162, HIT_RATE: 0.0291699, ARHR_ALL_HITS: 0.0094734, NOVELTY: 0.0053279, AVERAGE_POPULARITY: 0.0899813, DIVERSITY_MEAN_INTER_LIST: 0.1977629, DIVERSITY_HERFINDAHL: 0.9197744, COVERAGE_ITEM: 0.0514163, COVERAGE_ITEM_HIT: 0.0006712, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.0234183, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0015370, SHANNON_ENTROPY: 4.1614985, RATIO_DIVERSITY_HERFINDAHL: 0.9201071, RATIO_DIVERSITY_GINI: 0.0044348, RATIO_SHANNON_ENTROPY: 0.3205856, RATIO_AVERAGE_POPULARITY: 0.7503229, RATIO_NOVELTY: 0.3119123, 

LightGCNRecommender_PyTorch: New best model found! Updating.
LightGCNRecommender_PyTorch: Epoch 1 of 10. El

  0%|          | 0/96 [00:00<?, ?it/s]

LightGCNRecommender_PyTorch: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 10.67 sec. Users per second: 980
LightGCNRecommender_PyTorch: CUTOFF: 10 - PRECISION: 0.0011764, PRECISION_RECALL_MIN_DEN: 0.0018660, RECALL: 0.0014319, MAP: 0.0004669, MAP_MIN_DEN: 0.0008025, MRR: 0.0046022, NDCG: 0.0017907, F1: 0.0012916, HIT_RATE: 0.0114767, ARHR_ALL_HITS: 0.0046354, NOVELTY: 0.0060832, AVERAGE_POPULARITY: 0.0344184, DIVERSITY_MEAN_INTER_LIST: 0.1031190, DIVERSITY_HERFINDAHL: 0.9103109, COVERAGE_ITEM: 0.0272520, COVERAGE_ITEM_HIT: 0.0004922, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.0092138, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0007891, SHANNON_ENTROPY: 3.7639601, RATIO_DIVERSITY_HERFINDAHL: 0.9106401, RATIO_DIVERSITY_GINI: 0.0022769, RATIO_SHANNON_ENTROPY: 0.2899608, RATIO_AVERAGE_POPULARITY: 0.2870033, RATIO_NOVELTY: 0.3561317, 

LightGCNRecommender_PyTorch: Epoch 2 of 10. Elapsed time 45.99 sec


...

LightGCNRecommender_PyTorch: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 11.79 sec. Users per second: 887
LightGCNRecommender_PyTorch: CUTOFF: 10 - PRECISION: 0.0237089, PRECISION_RECALL_MIN_DEN: 0.0436706, RECALL: 0.0374804, MAP: 0.0079115, MAP_MIN_DEN: 0.0139267, MRR: 0.0651570, NDCG: 0.0330831, F1: 0.0290449, HIT_RATE: 0.1959640, ARHR_ALL_HITS: 0.0716936, NOVELTY: 0.0043866, AVERAGE_POPULARITY: 0.5943669, DIVERSITY_MEAN_INTER_LIST: 0.1785613, DIVERSITY_HERFINDAHL: 0.9178544, COVERAGE_ITEM: 0.0139616, COVERAGE_ITEM_HIT: 0.0007607, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.1573249, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0006237, SHANNON_ENTROPY: 3.8435777, RATIO_DIVERSITY_HERFINDAHL: 0.9181864, RATIO_DIVERSITY_GINI: 0.0017997, RATIO_SHANNON_ENTROPY: 0.2960943, RATIO_AVERAGE_POPULARITY: 4.9562191, RATIO_NOVELTY: 0.2568033, 

LightGCNRecommender_PyTorch: New best model found! Updating.
LightGCNRecommender_PyTorch: Epoch 8 of 10. Ela

  0%|          | 0/96 [00:00<?, ?it/s]

LightGCNRecommender_PyTorch: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 11.10 sec. Users per second: 942
LightGCNRecommender_PyTorch: CUTOFF: 10 - PRECISION: 0.0250287, PRECISION_RECALL_MIN_DEN: 0.0471382, RECALL: 0.0406985, MAP: 0.0067316, MAP_MIN_DEN: 0.0118110, MRR: 0.0515251, NDCG: 0.0315017, F1: 0.0309957, HIT_RATE: 0.2047628, ARHR_ALL_HITS: 0.0588407, NOVELTY: 0.0042044, AVERAGE_POPULARITY: 0.6361277, DIVERSITY_MEAN_INTER_LIST: 0.1864565, DIVERSITY_HERFINDAHL: 0.9186439, COVERAGE_ITEM: 0.0134694, COVERAGE_ITEM_HIT: 0.0008055, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.1643888, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0006258, SHANNON_ENTROPY: 3.8562413, RATIO_DIVERSITY_HERFINDAHL: 0.9189761, RATIO_DIVERSITY_GINI: 0.0018056, RATIO_SHANNON_ENTROPY: 0.2970698, RATIO_AVERAGE_POPULARITY: 5.3044484, RATIO_NOVELTY: 0.2461376, 

LightGCNRecommender_PyTorch: Epoch 9 of 10. Elapsed time 3.53 min


  0%|          | 0/96 [00:00<?, ?it/s]

LightGCNRecommender_PyTorch: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 10.90 sec. Users per second: 959
LightGCNRecommender_PyTorch: CUTOFF: 10 - PRECISION: 0.0229055, PRECISION_RECALL_MIN_DEN: 0.0426619, RECALL: 0.0366912, MAP: 0.0059526, MAP_MIN_DEN: 0.0103588, MRR: 0.0464719, NDCG: 0.0283351, F1: 0.0282039, HIT_RATE: 0.1895562, ARHR_ALL_HITS: 0.0525920, NOVELTY: 0.0042645, AVERAGE_POPULARITY: 0.5867594, DIVERSITY_MEAN_INTER_LIST: 0.1761714, DIVERSITY_HERFINDAHL: 0.9176155, COVERAGE_ITEM: 0.0128429, COVERAGE_ITEM_HIT: 0.0008502, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.1521806, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0006162, SHANNON_ENTROPY: 3.8367207, RATIO_DIVERSITY_HERFINDAHL: 0.9179473, RATIO_DIVERSITY_GINI: 0.0017780, RATIO_SHANNON_ENTROPY: 0.2955660, RATIO_AVERAGE_POPULARITY: 4.8927829, RATIO_NOVELTY: 0.2496605, 

LightGCNRecommender_PyTorch: Epoch 10 of 10. Elapsed time 3.92 min
LightGCNRecommender_PyTorch: Terminating 

In [17]:
early_stopping_params = {
    "validation_every_n": 1,
    "stop_on_validation": True,
    "evaluator_object": evaluator,
    "lower_validations_allowed": 5,
    "validation_metric": "MAP",
    "epochs": 500
}


def objective_function_lightgcn(trial):

    recommender_instance = LightGCNRecommender_PyTorch(URM_train)
    recommender_instance.fit(
        num_layers = trial.suggest_int('num_layers', 1, 10),
        embedding_size = trial.suggest_int('embedding_size', 5, 100, log=True),
        learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True),
        optimizer = trial.suggest_categorical('optimizer', ["adam", "sgd", "rmsprop", "adagrad"]),
        l2_reg = trial.suggest_float('l2_reg', 1e-5, 100, log=True),
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024]),
        **early_stopping_params
    )
    
    epochs = recommender_instance.get_early_stopping_final_epochs_dict()["epochs"]
    trial.set_user_attr("epochs", epochs) 
        
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10, "MAP"]

In [18]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='tuning_lightgcn_pytorch',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_lightgcn, n_trials=10)

[I 2023-12-24 10:50:41,503] Using an existing study with name 'tuning_lightgcn_pytorch' instead of creating a new one.


LightGCNRecommender_PyTorch: URM Detected 627 ( 4.8%) users with no interactions.
LightGCNRecommender_PyTorch: URM Detected 247 ( 1.1%) items with no interactions.
LightGCNRecommender_PyTorch: Built LightGCN Model, starting to train


  0%|          | 0/48 [00:00<?, ?it/s]

LightGCNRecommender_PyTorch: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 10.42 sec. Users per second: 1003
LightGCNRecommender_PyTorch: CUTOFF: 10 - PRECISION: 0.0009373, PRECISION_RECALL_MIN_DEN: 0.0017958, RECALL: 0.0014980, MAP: 0.0002894, MAP_MIN_DEN: 0.0005604, MRR: 0.0028731, NDCG: 0.0013453, F1: 0.0011531, HIT_RATE: 0.0092770, ARHR_ALL_HITS: 0.0028837, NOVELTY: 0.0066489, AVERAGE_POPULARITY: 0.0256999, DIVERSITY_MEAN_INTER_LIST: 0.9963510, DIVERSITY_HERFINDAHL: 0.9996256, COVERAGE_ITEM: 0.3919542, COVERAGE_ITEM_HIT: 0.0038036, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.0074478, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.1488290, SHANNON_ENTROPY: 12.0767341, RATIO_DIVERSITY_HERFINDAHL: 0.9999871, RATIO_DIVERSITY_GINI: 0.4294311, RATIO_SHANNON_ENTROPY: 0.9303446, RATIO_AVERAGE_POPULARITY: 0.2143023, RATIO_NOVELTY: 0.3892484, 

LightGCNRecommender_PyTorch: New best model found! Updating.
LightGCNRecommender_PyTorch: Epoch 1 of 500. 

  0%|          | 0/48 [00:00<?, ?it/s]

...

  0%|          | 0/24 [00:00<?, ?it/s]

LightGCNRecommender_PyTorch: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 10.78 sec. Users per second: 970
LightGCNRecommender_PyTorch: CUTOFF: 10 - PRECISION: 0.0287204, PRECISION_RECALL_MIN_DEN: 0.0519826, RECALL: 0.0443223, MAP: 0.0113324, MAP_MIN_DEN: 0.0202869, MRR: 0.0882480, NDCG: 0.0435267, F1: 0.0348550, HIT_RATE: 0.2243688, ARHR_ALL_HITS: 0.0996061, NOVELTY: 0.0040432, AVERAGE_POPULARITY: 0.6550198, DIVERSITY_MEAN_INTER_LIST: 0.3808432, DIVERSITY_HERFINDAHL: 0.9380807, COVERAGE_ITEM: 0.0315926, COVERAGE_ITEM_HIT: 0.0012977, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.1801290, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0015217, SHANNON_ENTROPY: 4.6231549, RATIO_DIVERSITY_HERFINDAHL: 0.9384200, RATIO_DIVERSITY_GINI: 0.0043906, RATIO_SHANNON_ENTROPY: 0.3561499, RATIO_AVERAGE_POPULARITY: 5.4619833, RATIO_NOVELTY: 0.2367011, 

LightGCNRecommender_PyTorch: Epoch 77 of 500. Elapsed time 16.38 min


  0%|          | 0/24 [00:00<?, ?it/s]

LightGCNRecommender_PyTorch: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 10.85 sec. Users per second: 964
LightGCNRecommender_PyTorch: CUTOFF: 10 - PRECISION: 0.0288734, PRECISION_RECALL_MIN_DEN: 0.0523631, RECALL: 0.0446773, MAP: 0.0113038, MAP_MIN_DEN: 0.0202151, MRR: 0.0881150, NDCG: 0.0435847, F1: 0.0350774, HIT_RATE: 0.2259946, ARHR_ALL_HITS: 0.0994224, NOVELTY: 0.0040405, AVERAGE_POPULARITY: 0.6575937, DIVERSITY_MEAN_INTER_LIST: 0.3797456, DIVERSITY_HERFINDAHL: 0.9379709, COVERAGE_ITEM: 0.0314584, COVERAGE_ITEM_HIT: 0.0013425, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.1814343, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0015092, SHANNON_ENTROPY: 4.6145017, RATIO_DIVERSITY_HERFINDAHL: 0.9383102, RATIO_DIVERSITY_GINI: 0.0043545, RATIO_SHANNON_ENTROPY: 0.3554833, RATIO_AVERAGE_POPULARITY: 5.4834456, RATIO_NOVELTY: 0.2365462, 

LightGCNRecommender_PyTorch: Convergence reached! Terminating at epoch 78. Best value for 'MAP' at epoch 73 

[I 2023-12-24 15:41:36,128] Trial 69 finished with value: 0.011363669830339644 and parameters: {'num_layers': 1, 'embedding_size': 22, 'learning_rate': 7.777821859131347e-05, 'optimizer': 'adam', 'l2_reg': 0.0002944342784337146, 'batch_size': 512}. Best is trial 63 with value: 0.01156761212518667.


In [19]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [20]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [21]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [22]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_lightgcn_pytorch.json', 'w') as params_file:
        params = study.best_params
        params['epochs'] = study.best_trial.user_attrs['epochs']
        json.dump(params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_lightgcn_pytorch.json', 
            'tuning_results/best_params_lightgcn_pytorch.json', 
            'LightGCN PyTorch tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/best_params_lightgcn_pytorch.json',
        '/kaggle/working/best_params_lightgcn_pytorch.json'
    )

In [23]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_lightgcn_pytorch.db',
        'LightGCN tuning db updated results (from kaggle notebook)'
    )